In [32]:
import arxiv
from datetime import datetime, timedelta, timezone, time, date
from zoneinfo import ZoneInfo

In [33]:
def last_arxiv_new_window(now_et=None):
    ET = ZoneInfo("America/New_York")
    if now_et is None:
        now_et = datetime.now(ET)
    # announcement candidate: today if after 20:00 ET, else yesterday
    candidate = now_et.date() if now_et.time() >= time(20, 0) else (now_et.date() - timedelta(days=1))
    # if candidate falls on Fri/Sat, back up to the last announcement day
    while candidate.weekday() in (4, 5):  # Fri=4, Sat=5
        candidate -= timedelta(days=1)

    wd = candidate.weekday()
    if wd == 0:  # Monday announcement -> window: Fri 14:00 -> Mon 14:00 (3 days)
        start_et = datetime.combine(candidate - timedelta(days=3), time(14, 0), tzinfo=ET)
        end_et   = datetime.combine(candidate, time(14, 0), tzinfo=ET)
    elif wd == 6: # Sunday announcement -> window: Thu 14:00 -> Fri 14:00 (see schedule)
        start_et = datetime.combine(candidate - timedelta(days=3), time(14, 0), tzinfo=ET)  # Thu 14:00
        end_et   = datetime.combine(candidate - timedelta(days=2), time(14, 0), tzinfo=ET)  # Fri 14:00
    else:
        # Tue/Wed/Thu announcements -> window is (candidate-1 day 14:00) -> candidate 14:00
        start_et = datetime.combine(candidate - timedelta(days=1), time(14, 0), tzinfo=ET)
        end_et   = datetime.combine(candidate, time(14, 0), tzinfo=ET)

    # convert to UTC for comparing against API datetimes
    start_utc = start_et.astimezone(ZoneInfo("UTC"))
    end_utc   = end_et.astimezone(ZoneInfo("UTC"))
    return start_utc, end_utc, candidate

start, end, announcement_date = last_arxiv_new_window()
print("mailing window (UTC):", start, "→", end, " (announcement date:", announcement_date, ")")

# Query arXiv API (may still lag; increase max_results)
search = arxiv.Search(
    query="cat:quant-ph",
    max_results=2000,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending,
)
client = arxiv.Client()
new = []
for r in client.results(search):
    if start <= r.published < end:
        new.append(r)

print("Found", len(new), "entries via API for that window (expected: match site/RSS).")
for r in new:
    print(r.entry_id, r.published, r.title)

mailing window (UTC): 2025-09-30 18:00:00+00:00 → 2025-10-01 18:00:00+00:00  (announcement date: 2025-10-01 )


UnexpectedEmptyPageError: Page of results was unexpectedly empty (https://export.arxiv.org/api/query?search_query=cat%3Aquant-ph&id_list=&sortBy=submittedDate&sortOrder=descending&start=400&max_results=100)

In [28]:
# Arxiv's daily cutoff is 00:00 UTC
now = datetime.now(timezone.utc)
today_utc = datetime(now.year, now.month, now.day, tzinfo=timezone.utc)

# Yesterday's UTC window
start = today_utc - timedelta(days=2)
end = today_utc - timedelta(days=1)

print("Collecting new submissions from", start, "to", end)

search = arxiv.Search(
    query="cat:quant-ph",
    max_results=200,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending,
)

client = arxiv.Client()

new_submissions = []
for result in client.results(search):
    # Important: check the first submission time (published)
    if start <= result.published < end:
        new_submissions.append(result)

# Print them out
for res in new_submissions:
    print(res.entry_id, res.published, res.title)

http://arxiv.org/abs/2509.26623v1 2025-09-30 17:57:08+00:00 Quantum Simulation of Random Unitaries from Clebsch-Gordan Transforms
http://arxiv.org/abs/2509.26620v1 2025-09-30 17:55:55+00:00 Amplified response of cavity-coupled quantum-critical systems
http://arxiv.org/abs/2509.26615v1 2025-09-30 17:53:55+00:00 Non-Hermitian comb effect in coupled clean and quasiperiodic chains
http://arxiv.org/abs/2509.26602v1 2025-09-30 17:49:09+00:00 Quantum Optics and Quantum Electrodynamics of Strong Field Processes
http://arxiv.org/abs/2509.26574v2 2025-09-30 17:34:03+00:00 Probing the Critical Point (CritPt) of AI Reasoning: a Frontier Physics Research Benchmark
http://arxiv.org/abs/2509.26570v1 2025-09-30 17:33:03+00:00 Electrical Readout of Spin Environments in Diamond for Quantum Sensing
http://arxiv.org/abs/2509.26533v1 2025-09-30 17:07:37+00:00 Topology of the Aharonov-Bohm effect in different reference frames
http://arxiv.org/abs/2509.26504v1 2025-09-30 16:45:58+00:00 Structure-preserving n

In [34]:
# pip install feedparser
import feedparser
from datetime import datetime, timezone

feed = feedparser.parse("https://rss.arxiv.org/rss/quant-ph")
for e in feed.entries:
    # e.link looks like "https://arxiv.org/abs/2510.00056"
    arxiv_id = e.link.rsplit("/", 1)[-1]
    # published_parsed -> time.struct_time (UTC)
    if hasattr(e, "published_parsed"):
        pub = datetime(*e.published_parsed[:6], tzinfo=timezone.utc)
    else:
        pub = None
    print(arxiv_id, pub, e.title)


2510.00056 2025-10-02 04:00:00+00:00 Evaluating noises of boson sampling with statistical benchmark methods
2510.00064 2025-10-02 04:00:00+00:00 Operator algebra of the information-disturbance tradeoff in quantum measurements
2510.00108 2025-10-02 04:00:00+00:00 Passive detection of Schwinger boson dynamics via a qubit
2510.00132 2025-10-02 04:00:00+00:00 Complexity and hardness of random peaked circuits
2510.00145 2025-10-02 04:00:00+00:00 Approximate Quantum State Preparation with Tree-Based Bayesian Optimization Surrogates
2510.00157 2025-10-02 04:00:00+00:00 The non-stabilizerness cost of quantum state estimation
2510.00168 2025-10-02 04:00:00+00:00 Query-Optimal Estimation of Unitary Channels via Pauli Dimensionality
2510.00171 2025-10-02 04:00:00+00:00 Quantum reservoir computing using Jaynes-Cummings model
2510.00199 2025-10-02 04:00:00+00:00 Practical Quantum Clock Synchronization Using Weak Coherent Pulses
2510.00203 2025-10-02 04:00:00+00:00 A Review of Software for Designing